In [2]:
!pip3 install streamlit

In [1]:
import pandas as pd
import re
import pymorphy2

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('lenta-ru-train.csv')

In [3]:
df.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


In [4]:
df['text'] = df['text'].str.replace('/','')

In [5]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = '[^А-Яа-я0-9]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list

In [6]:
df['text_clean'] = remove_trash(df['text'])

expected string or bytes-like object


In [7]:
df = df[~df.text_clean.isna()].copy()

In [ ]:
df.shape

In [8]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [9]:
stop_russian = stopwords.words('russian')

In [10]:
len(stop_russian)

151

In [11]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=15000 )

In [12]:
df.tail()

,title,text,topic,topic_label,text_clean
206632,Бывшую вице-президента Tiffany обвинили в милл...,Бывшей вице-президенту ювелирной компании Tiff...,Экономика,0,Бывшей вице-президенту ювелирной компании Tiff...
206633,Минфин предложил увеличить штрафы за незаконны...,Минфин предлагает увеличить размер штрафа для ...,Экономика,0,Минфин предлагает увеличить размер штрафа для ...
206634,Потребительская уверенность россиян вернулась ...,Индекс потребительской уверенности россиян в п...,Экономика,0,Индекс потребительской уверенности россиян в п...
206635,Тарпищеву дали американскую визу,Капитан женской сборной России по теннису Шами...,Спорт,1,Капитан женской сборной России по теннису Шами...
206636,УЕФА пригрозил России исключением из числа уча...,Союз европейских футбольных ассоциаций (УЕФА) ...,Спорт,1,Союз европейских футбольных ассоциаций (УЕФА) ...


In [13]:
text = text_transformer.fit_transform(df['text_clean'])

In [17]:
df.topic.value_counts()

Экономика          63571
Спорт              51679
Культура           43071
Наука и техника    42384
Бизнес              5931
Name: topic, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.3, random_state=1912)

In [19]:
print(X_train.shape,X_test.shape)

(144645, 15000) (61991, 15000)


In [20]:
from sklearn import tree

In [21]:
clf = tree.DecisionTreeClassifier()

In [22]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [24]:
pred = clf.predict(X_test)

In [26]:
import sklearn

In [27]:
print('Качество модели по метрике F1', sklearn.metrics.f1_score(y_test,pred,average='macro'))

Качество модели по метрике F1 0.7899551218632959


In [28]:
morph = pymorphy2.MorphAnalyzer()

In [30]:
%%time #првоерить время выполнения функции
# функция лемматизации - приводит в 1 форму
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:   #удаляет слова, которые меньше == 3 символа
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 22.2 µs


In [31]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: user 1h 12min 9s, sys: 40.8 s, total: 1h 12min 49s
Wall time: 1h 13min 48s


In [33]:
df[['text_clean','text_clean_normal']]

,text_clean,text_clean_normal
0,"Глава Росстата Владимир Соколин заявил, что в ...","глава росстат владимир соколиный заявил, январ..."
1,У российских кинематографистов появится новый...,российский кинематографист появиться новый про...
2,"Медицинский штаб миланского ""Интера"" обнародов...","медицинский штаб миланский ""интера"" обнародова..."
3,"Премии ""Книга года"" в 13 номинациях вручены на...","премия ""книга года"" номинация вручить московск..."
4,Сборная Португалии сыграла вничью с командой И...,сборная португалия сыграть вничью команда испа...
...,...,...
206632,Бывшей вице-президенту ювелирной компании Tiff...,бывший вице-президент ювелирный компания tiffa...
206633,Минфин предлагает увеличить размер штрафа для ...,минфин предлагать увеличить размер штраф должн...
206634,Индекс потребительской уверенности россиян в п...,индекс потребительский уверенность россиянин п...
206635,Капитан женской сборной России по теннису Шами...,капитан женский сборная россия теннис шамиль т...


### качество модели после лемматизации

In [34]:
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic_label'], test_size=0.3, random_state=1912)

In [35]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)
print('Качество модели по метрике F1 после лемматизации', sklearn.metrics.f1_score(y_test,pred_norm,average='macro'))

Качество модели по метрике F1 после нлемматизации 0.8122978184133182
